In [38]:
import pandas as pd;
import numpy as np;
import plotly.express as px
artists=pd.read_csv("data/artists.csv")


# Artist Dataset


In [39]:
artists.sample(5)

,id,followers,genres,name,popularity
925181,7JJE29ubbjgYfJOQFAcPqb,10.0,[],Piltch,0
151624,3xKvIIizveDU2e542VdHCT,6910.0,"['german rock', 'psychedelic blues-rock']",Bang,22
128667,04zQ0PKrh3c5iYkmUAGm47,10.0,[],西瓜,0
914961,3b6Xu46myEwqKjHhCb5PFt,945.0,[],DJ Raff & Solo Di Medina,26
564074,0hFKM3Tcm1n3pB4NnLPMjZ,173.0,[],Aves,20


# The most popular genre

In [40]:
artists["TmpGenres"]=artists["genres"].apply(lambda x: x.strip('][').split(','))#convert arrays
allGeners=np.unique(np.concatenate(artists["TmpGenres"]))
allGeners[0]="unknown"
allGeners# all geners


array(['unknown', ' "black \'n\' roll"', ' "children\'s choir"', ...,
       "'zouk'", "'zurich indie'", "'zydeco'"], dtype='<U41')

In [41]:
generesCount=dict.fromkeys(allGeners, 0)
for g in artists["TmpGenres"]:
    for k in g:
        if(k in generesCount.keys()):
            generesCount[k]= generesCount[k] + 1

sorted(generesCount.items(), key=lambda x:x[1], reverse=True)

#Generes counted



[(" 'rock'", 564),
 (" 'pop'", 531),
 (" 'electro house'", 487),
 (" 'rap'", 449),
 (" 'indie rock'", 442),
 (" 'hip hop'", 425),
 (" 'pop rap'", 418),
 (" 'modern rock'", 409),
 ("'calming instrumental'", 406),
 (" 'tropical'", 399),
 (" 'pop rock'", 389),
 (" 'urban contemporary'", 380),
 (" 'country rock'", 371),
 (" 'indian classical'", 366),
 (" 'latin rock'", 363),
 ("'dance pop'", 362),
 ("'k-pop'", 362),
 (" 'pop dance'", 360),
 ("'adult standards'", 359),
 (" 'r&b'", 353),
 (" 'rock en espanol'", 352),
 (" 'trap'", 351),
 ("'background music'", 348),
 (" 'regional mexican'", 347),
 (" 'stomp and holler'", 347),
 ("'alternative r&b'", 347),
 (" 'post-teen pop'", 340),
 (" 'tropical house'", 340),
 (" 'underground hip hop'", 336),
 ("'alt z'", 334),
 ("'alternative metal'", 334),
 (" 'electropop'", 332),
 (" 'soft rock'", 329),
 (" 'edm'", 328),
 ("'latin'", 327),
 (" 'indie folk'", 322),
 (" 'mellow gold'", 321),
 ("'french hip hop'", 321),
 ("'dutch hip hop'", 320),
 (" 'folk 

In [49]:
#group in macro-category
myGeneres=generesCount.keys()
macroGeneres=dict.fromkeys(["alternative","rock","metal","pop","rap","punk","jazz","latin","reggae","soul","polka","country","electronic","funk","hip hop","r&b","folk","house","techno","trance","indie","blues","instrument","electronic","disco","deep","deathcore","hardcore","core","wave","other"],0)

for gen in myGeneres:
    founed=False
    for m in macroGeneres.keys():
        if gen.find(m)!=-1:
            founed=True
            macroGeneres[m]+=1
            break

#we classify only the general music category (rock, pop, metal ...)
macroGeneres

{'alternative': 110,
 'rock': 720,
 'metal': 705,
 'pop': 881,
 'rap': 418,
 'punk': 264,
 'jazz': 259,
 'latin': 50,
 'reggae': 98,
 'soul': 66,
 'polka': 2,
 'country': 62,
 'electronic': 135,
 'funk': 73,
 'hip hop': 316,
 'r&b': 45,
 'folk': 305,
 'house': 152,
 'techno': 108,
 'trance': 59,
 'indie': 790,
 'blues': 77,
 'instrument': 41,
 'disco': 22,
 'deep': 65,
 'deathcore': 18,
 'hardcore': 106,
 'core': 73,
 'wave': 51,
 'other': 0}

In [50]:
generesDf={'Generes':macroGeneres.keys(),'Number':macroGeneres.values()}

macroGeneresDF=pd.DataFrame.from_dict(generesDf)
macroGeneresDF=macroGeneresDF.sort_values(by="Number",ascending=False)

In [52]:
fig=px.histogram(macroGeneresDF,x="Generes",y="Number")
fig.layout["xaxis"]["title"]="Generes"
fig.layout["yaxis"]["title"]="Number of Artists"
fig.show()